# Group Members:

*   Name 1
*   Name 2
*   Name 3

# Lab 3 Assignment (Part 2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "PUT_YOUR_PATH"

In [ ]:
# Install if required
# !python -m pip install opencv-python

In [ ]:
import numpy as np
from PIL import Image 
import os
import cv2 as cv
import matplotlib.pyplot as plt
from math import sqrt
from IPython.display import clear_output, display

#### **1. Complete the code required in the cells by following the comments provided in the code.**

**Read the images**

In [ ]:
# Define the images directory
images_dir = os.path.abspath("../images")

# Read all the required images
ginevra = Image.open(os.path.join(images_dir, "ginevra.png"))
lisa = Image.open(os.path.join(images_dir, "lisa.png"))
mask = Image.open(os.path.join(images_dir, "mask.png"))

# Convert all the images to 2D
u1 = np.array(ginevra, dtype = float)[:, :, 0]
u2 = np.array(lisa, dtype = float)[:, :, 0]
mask = np.array(mask, dtype = float) / 255

# Show all the images
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 8))
ax1.imshow(u1, cmap = "gray")
ax1.set_title("Ginevra")
ax2.imshow(u2, cmap = "gray")
ax2.set_title("Lisa")
ax3.imshow(mask, cmap = "gray")
ax3.set_title("Mask")
plt.show()

**Apply filters to images**

Learn more about Gaussian blurring [here](https://docs.opencv.org/master/d4/d13/tutorial_py_filtering.html)

In [ ]:
u1 = cv.GaussianBlur(src = u1, ksize = (9, 9), sigmaX = 2, borderType = cv.BORDER_REFLECT)
u2 = cv.GaussianBlur(src = u2, ksize = (11, 11), sigmaX = 4, borderType = cv.BORDER_REFLECT)

**Crude composition**

In [ ]:
# TODO: generate an image which is u1 when mask = 1, and u2 when mask = 0
ucomp = None

# Show the composed image
plt.figure(figsize = (12, 8))
plt.imshow(ucomp, cmap = "gray")

**Guiding vector field**

We compose the images, but at the gradient level!

Let:

$\nabla^+ u_{1,ij}= (\nabla^+_i u_{1,ij}, \nabla^+_j u_{1,ij})$

$\nabla^+ u_{2,ij}= (\nabla^+_i u_{2,ij}, \nabla^+_j u_{2,ij})$

for $i=1,2,\cdots M,j=1,2,\cdots N.$

$\quad$

Then 
\begin{align*}
v_{ij} &= m_{ij}\nabla^+ u_{1,ij} + (1-m_{ij})\nabla^+ u_{2,ij}\\
&=(m_{ij}\nabla^+_i u_{1,ij} + (1-m_{ij})\nabla^+_i u_{2,ij}\quad ,\quad m_{ij}\nabla^+_j u_{1,ij} + (1-m_{ij})\nabla^+_j u_{2,ij})\\
&=(v_{1,ij} ,v_{2,ij})
\end{align*}




In [ ]:
def im_fwd_gradient(image: np.matrix):
    """
    Discrete gradient of an image using forward differences, with homogeneous Neuman boundary conditions.

    :param u: image (MxN)
            
    :return gradu_j: partial derivative in the j (rows) direction (also x direction)
    :return gradu_i: partial derivative in the i (cols) direction (also y direction)
    """
    # Get the size of the image
    image_shape = image.shape
    
    # Calculate both gradients
    gradu_j = np.append(np.diff(image, axis = 1), np.zeros((image_shape[0], 1)), axis = 1)
    gradu_i = np.append(np.diff(image, axis = 0), np.zeros((1, image_shape[1])), axis = 0)
    return gradu_i, gradu_j

In [ ]:
# TODO: Compute the gradients of u1 and u2, and define a new gradient [vi,vj] 
#       which corresponds to the gradient of u1 when mask = 1, and the gradient 
#       of u2 when mask = 0

# Compute the gradient of the images

# Calculate v
vi, vj = None, None

In [ ]:
plt.figure(figsize = (24, 8))
plt.subplot(1,2,1)
plt.imshow(vi, cmap = "gray")
plt.subplot(1,2,2)
plt.imshow(vj, cmap = "gray")

Find $u$ that minimizes the energy function:
$$E(u) = \overbrace{\dfrac{1}{2}\sum_{i = 1}^N\sum_{j = 1}^M |\nabla^+u_{ij} - v_{ij}|_{\mathbb R^2}^2}^{\text{gradients similar to $v$}} +
\overbrace{\dfrac{1}{2}\sum_{i = 1}^N\sum_{j = 1}^M \beta_{ij}(u_{ij} - u_{2,ij})^2}^{\text{$u$ is $u_{2}$ outside the mask}}$$

where:

$\beta_{ij} = \beta_{0} (1-m_{ij})$

**Conjugate gradient set up**

With

\begin{align*}
E(u) &= \overbrace{\dfrac{1}{2}\langle\nabla^+u - v, \nabla^+u - v\rangle_{Y}}^{\text{gradients similar to $v$}} +
\overbrace{\dfrac{1}{2}\langle B(u - u_{2}), u - u_{2}\rangle_{X}}^{\text{$u$ is $u_{2}$ outside the mask}}\\
&\\
&\\
\nabla E(u) &= -div^{-}(\nabla^{+}u - v) + B(u - u_{2})
\end{align*}

In [ ]:
def im_bwd_divergence(gradient_i: np.matrix,
                      gradient_j: np.matrix):
    """
    Discrete divergence of a vector field using backwards differences. 
    This is the negative transpose of the im_fwd_gradient
    
    :param gradient_i: component of g in the direction j (rows) (also x direction)
    :param gradient_j: component of g in the direction i (cols) (also y direction)
    
    :return divg: backwards divergence of g
    """
    # Backwards j partial derivative of gradient_j
    gradient_j[:, gradient_j.shape[1] - 1] = 0
    divg = np.diff(np.append(np.zeros((gradient_j.shape[0], 1)), gradient_j, axis = 1), axis = 1)
    
    # Backwards i partial derivative of gradient_i
    gradient_i[gradient_i.shape[0] - 1, :] = 0
    divg = np.diff(np.append(np.zeros((1, gradient_i.shape[1])), gradient_i, axis = 0), axis = 0) + divg
    
    return divg

**Define the variable term**

\begin{align*}
\nabla E(u) &= -div^{-}(\nabla^{+}u - v) + B(u - u_{2})\\
&= \overbrace{[-div^{-}(\nabla^{+}u) + B*u]}^{\text{variable term}}-\overbrace{[-div^{-}(v) + B*u_{2}]}^{\text{constant term}}\\
\end{align*}

With

$\begin{align*}
Ax&= -div^{-}(\nabla^{+}u) + B*u\\
b&= -div^{-}(v) + B*u_{2}
\end{align*}$


$$\nabla E(u) = Ax - b$$

In [ ]:
def poisson_linear_operator(u, beta):
    """
    Discrete laplacian of an image using a forward gradient and 
    a backward divergence. It computes the Laplacian inside a region indicated by a 
    mask (mask = 1). It assumes u(x) = 0 for x outside the mask (homogeneous
    Dirichlet boundary conditions). The interface is compatible with
    conjugate_gradient and gradient_descent.

    lap = im_laplacian_mask(u,prms)

    param u: image (MxN)
    param prms: structure with one field, prms.mask, a (MxN) binary image

    :return lap: laplacian
    """
    gu_i , gu_j = im_fwd_gradient(u)
    lapu = (-1) * im_bwd_divergence(gu_i, gu_j)
    return lapu + beta * u

In [ ]:
def conjugate_gradient(callback,
                       b: np.matrix,
                       callback_params: dict,
                       initial_condition: np.matrix,
                       tolerance: float,
                       max_iters: int,
                       fig = None,
                       ax = None):
    """
    implementation of the conjugate gradient algorithm for the minimization of quadratic problems 

       f(x) = 1/2 x'Ax - bx. 

    It uses function handles. It requires a handle to a Python function that implements the product of matrix A with x.

    :param callback: handle (pointer) to a matlab function implementing the product with matrix A. 
    :param callback_params: dictionary containing the params for the callback functions
    :param b: vector b, can be in matrix form (MxN)
    :param initial_conditions: initial condition, same dimensions as b (MxN)
    :param max_iters: maximum number of iterations
    :param tolerance: tolerance for the stopping condition (it stop when the norm of the gradient is below the tolerance)

    :return x: value found (MxN)
    :return fs: evolution of the target function (total_iters x 1 vector)
    """

    # NOTE: Copy and paste the function you already implemented in the first part of the exercise

#### **2. Once you have completed all the code above, now implemented into the function lisa_ginevra_test and explain with your own words the main steps of the algorithm**

**Define the constant term**

\begin{align*}
\nabla E(u) &= -div^{-}(\nabla^{+}u - v) + B(u - u_{2})\\
&= \overbrace{[-div^{-}(\nabla^{+}u) + B*u]}^{\text{variable term}}-\overbrace{[-div^{-}(v) + B*u_{2}]}^{\text{constant term}}\\
\end{align*}

With

$\begin{align*}
Ax&= -div^{-}(\nabla^{+}u) + B*u\\
b&= -div^{-}(v) + B*u_{2}
\end{align*}$


$$\nabla E(u) = Ax - b$$

Recall that:
\begin{align*}
v_{ij} &= m_{ij}\nabla^+ u_{1,ij} + (1-m_{ij})\nabla^+ u_{2,ij}\\
&=(m_{ij}\nabla^+_i u_{1,ij} + (1-m_{ij})\nabla^+_i u_{2,ij}\quad ,\quad m_{ij}\nabla^+_j u_{1,ij} + (1-m_{ij})\nabla^+_j u_{2,ij})\\
&=(v_{1,ij} ,v_{2,ij})
\end{align*}

In [ ]:
def lisa_ginevra_test(u1: np.matrix, 
                      u2: np.matrix,
                      mask: np.matrix):
    """
    uses Poisson editing to interchange the faces of Ginevra de' Benci and Lisa Gherardini
    
    :param u1: Ginevra's image
    :param u2: Lisa's image
    """
    # TODO: CRUDE COMPOSITION
    ucomp = None
    # TODO: GUIDING VECTOR FIELD
    vi, vj = None
    # Define beta_0 and calculate beta
    beta_0 = 20                            # TRY CHANGING
    beta = beta_0 * (1 - mask)

    # Calculate the right hand term with bounday data
    b = (-1) * im_bwd_divergence(vi, vj) + beta * u2

    # Define the parameters to be used when calculating the conjugate gradient
    initial_condition = np.zeros_like(u1)  # TRY CHANGING    
    tolerance = 1                          # TRY CHANGING
    max_iters = 1000                       # TRY CHANGING
    upns, fs = conjugate_gradient(callback = poisson_linear_operator,
                                  b = b,
                                  callback_params = {"beta": beta},
                                  initial_condition = initial_condition,
                                  tolerance = tolerance,
                                  max_iters = max_iters)
    
    # Plot
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows = 2, ncols = 2, figsize = (16, 8))
    ax1.imshow(u1, cmap = "gray")
    ax1.set_title("Ginevra")
    ax2.imshow(u2, cmap = "gray")
    ax2.set_title("Lisa")
    ax3.imshow(upns, cmap = "gray")
    ax3.set_title("composing gradients")
    ax4.imshow(ucomp, cmap = "gray")
    ax4.set_title("composing gray levels")
    plt.show()

In [ ]:
lisa_ginevra_test(u1, u2, mask)

#### **3. Run the function lisa_ginevra_test using different parameters and explain the differences**

#### **4. Take a picture of two different faces (u1 and u2) and apply Poisson editing (it will be better if the faces are from the two components of the group). Try using u1 as a background and the face of u2 and the way around. Notice that you will need to create your own mask.**